In [ ]:
#document_id	sentence_id	doc_start_index	doc_end_index	sentence_entity_start	sentence_entity_end	text_segment	semantic_type_id	entity_id

In [ ]:
import pandas as pd

In [ ]:
#df_checkpoint_mm_multi_sentences_671
df_result_outputs = pd.read_csv('df_checkpoint_bg_test_sentences_prompt_0928_v1_13b.tsv', sep='\t', header=0)
df_result_outputs.head()

In [ ]:
from ast import literal_eval
df_result_outputs['outputs'] = df_result_outputs['outputs'].apply(literal_eval)

In [ ]:
result_outputs = list(df_result_outputs['outputs'].values)

In [ ]:
result_outputs[0:10]

In [ ]:
def unique_occurences(occurences):
    # intilize a null list
    unique_list = []

    # traverse for all elements
    for x in occurences:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)

    return unique_list

In [ ]:
import re
def find_original_span(text, term, abbr=False):    
    found_original_text = ''
    
    search_text = text
    search_term = term
    if abbr and search_term in search_text:
        found_original_text = search_term
    else:       
        search_text = text.lower()
        search_term = term.lower()
        if search_term in search_text:
            found_original_text = search_term
        #else:
        #    found_original_text = fuzzy_ngram_match(term, text).lower()

    if found_original_text != '':
        print(found_original_text)
        occurrences_start = [m.start() for m in re.finditer(re.escape(found_original_text), search_text)]
        found_occurences = []
        for start_idx in occurrences_start:
            #start_idx = current_idx + text.lower()[current_idx:].index(found_original_text)
            end_idx = start_idx + len(found_original_text)
            found_occurences.append({
                'start_idx': start_idx,
                'end_idx': end_idx,
                'term': text[start_idx:end_idx]
            })
        return found_occurences
    else:
        return [{
            'start_idx': -1,
            'end_idx': -1,
            'term': 'no-match'
        }]

In [ ]:
import json
import re
def convert_output_from_comma_list(text, outputs, abbr=False):
    outputs = str(outputs)
    if ':' in outputs[0:150]:
        outputs = outputs[outputs.index(':')+1:]
    outputs_replaced = outputs.replace('*', '')
    #.replace('\\n', ',')
    term_lines = [term_line.strip() for term_line in outputs_replaced.split('\n')]
    terms_from_lines = [term_line.split(',') for term_line in term_lines]
    terms = [term.strip() for term_line in terms_from_lines for term in term_line]
    #print(f'terms: {terms}')
    original_terms = []
    for term in terms:
        if len(term.split(' ')) < 10: # longer terms are probably explanations
            original_term = find_original_span(text, term, abbr)
            if len(original_term) > 0 and original_term[0]['start_idx'] > -1:
                original_terms.extend(original_term)
    
    #todo: handle nested entities
    return unique_occurences(original_terms)

In [ ]:
def filter_translations(translation):
    if len(translation) == 0 or translation == 'N/A' or translation.startswith('C0') or '\\' in translation:
        return False
    return True

In [ ]:
import json
import re
def convert_translation_from_comma_list(outputs):
    outputs = str(outputs)
    if ':\n' in outputs[0:150]:
        outputs = outputs[outputs.index(':')+1:]
    outputs_replaced = outputs.replace('*', '')
    #.replace('\\n', ',')
    term_lines = [term_line.strip() for term_line in outputs_replaced.split('\n')]
    terms_from_lines = [term_line.split(',') for term_line in term_lines]
    terms = [term.strip() for term_line in terms_from_lines for term in term_line]
    #print(f'terms: {terms}')
    original_terms = []
    for term in terms:
        if len(term.split(' ')) < 10: # longer terms are probably explanations
            # handle term: translation
            if term.startswith('1. ') or term.startswith('2. ') or term.startswith('3. ') or term.startswith('4. ') or term.startswith('5. ') or term.startswith('6. ') or term.startswith('7. ') or term.startswith('8. ') or term.startswith('9. '):                
                term = term[3:]
                # 1. - beginning number
            
            if ':' in term:
                subterms = term.split(': ')
                if len(subterms) == 2:
                    original_terms.append(subterms[1])
            elif '-' in term:
                subterms = term.split(' - ') #"оплаква се" - complains
                # term - C4440639
                if len(subterms) == 2:
                    if not subterms[1].startswith('C'):
                        original_terms.append(subterms[1])
                    else:
                        original_terms.append(subterms[0])
            elif '(' in term and ')' in term:
                subterms = term.split('(')
                for subterm in subterms:
                    subterm = subterm.replace(')', '')
                    if subterm.startswith('C') or subterm.startswith('S'):
                        continue
                    x = re.search("[а-яА-Я]*", subterm)
                    if len(x[0]) > 0:
                        continue
                # (C0694467) - replace suffix
                #заболял (became ill)
                #S0002556 (became ill)
            else:
                original_terms.append(term)
    
    #todo: handle nested entities
    #return unique_occurences(original_terms)
    return [x for x in original_terms if filter_translations(x)]

In [ ]:
df_sentences = pd.read_csv('../MedMentions-master/MedMentions-master/bg_texts_30.txt', sep='\t', header=None)
df_sentences.columns = ['sentence']
df_sentences.head()

In [ ]:
texts = list(df_sentences['sentence'].values)

In [ ]:
result_original = []
results_translated = []

for text, outputs in zip(texts, result_outputs):    
    terms = convert_output_from_comma_list(text, outputs[0])
    print(terms)
    result_original.append(terms)
    #results_translated.append(outputs[1].split(','))
    results_translated.append(convert_translation_from_comma_list(outputs[1]))
    #combined = []
    #for output in outputs:
    #    combined += convert_output_from_comma_list(text, output)
    #result_original.append(combined)

In [ ]:
results_translated

In [ ]:
len(result_original)

In [ ]:
df_small_results = pd.DataFrame({'outputs': result_outputs, 'original': result_original, 'translated': results_translated}) #, 'medical_term': result_medical_terms})
df_small_results.head(30)

In [ ]:
df_subset = df_sentences#[0:1221] #.tail(600)

In [ ]:
df_outputs = df_small_results

In [ ]:
df_outputs['sentence'] = list(df_subset['sentence'].values)

In [ ]:
df_outputs.head()

In [ ]:
df_outputs['entities'] = df_outputs['original']

In [ ]:
df_outputs.iloc[0]['entities']

In [ ]:
result_entities = []

for index, row in df_outputs.iterrows():
    for entity, translation in zip(row['entities'], row['translated']):
        if entity['start_idx'] > -1:
            result_entities.append({
                'document_id': index,
                'sentence_id': 0,
                'doc_start_index': 0,
                'doc_end_index': len(row['sentence']),
                'sentence_entity_start': entity['start_idx'],
                'sentence_entity_end': entity['end_idx'],
                'text_segment': entity['term'],
                'translation': translation.lower()
            })
        

In [ ]:
df_result_entities = pd.DataFrame(result_entities)
df_result_entities.head()

In [ ]:
df_result_entities.to_csv('bg_test_0928v113b_llama_entities_nofuzzy.tsv', sep='\t', index=False)